In [1]:
import tensorflow as tf
import re

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# create config to use both gpus on wickerman machine:
# 1. Nvidia A40 (25GB memory allocation)
# 2. Nvidia RTX 3060 (11GB memory allocation)

gpus = tf.config.list_physical_devices('GPU')

tf.config.set_visible_devices(gpus[0], 'GPU') # use Use Nvidia A40 only

log_dev_conf_a40 = tf.config.LogicalDeviceConfiguration(
    memory_limit=45*1024 # 25 GB allocation for a40 GPU
)

tf.config.set_logical_device_configuration(
    gpus[0], # select GPU_0, i.e., Nvidia A40
    [log_dev_conf_a40] # apply 25GB config
)

# tf.config.set_visible_devices(gpus[1], 'GPU') # use RTX3060 only

# log_dev_conf_rtx3060 = tf.config.LogicalDeviceConfiguration(
#     memory_limit=12*1024 # 11 GB allocation for rtx3060 GPU
# )

# tf.config.set_logical_device_configuration(
#     gpus[1], # select GPU_1, i.e., Nvidia rtx3060
#     [log_dev_conf_rtx3060] # apply 11GB config
# )

# MultiGPU setup
# Create a MirroredStrategy.
# strategy = None
# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")

2024-09-02 23:16:04.557700: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 23:16:04.557721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 23:16:04.558585: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 23:16:04.562685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-02 23:16:05.104875: W tensorflow/compiler/tf2

2 Physical GPUs, 1 Logical GPU


2024-09-02 23:16:05.736321: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-02 23:16:05.737636: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-02 23:16:05.759943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import json
import os
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import Progbar
from tensorflow.keras.metrics import Mean
from pqdm.threads import pqdm
from tqdm import tqdm

In [3]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration,

# model_id = "google-t5/t5-base"
model_id = 'google/flan-t5-base'

# with strategy.scope():
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFT5ForConditionalGeneration.from_pretrained(model_id)

# tokenizer = AutoTokenizer.from_pretrained("models/v2")
# model = TFT5ForConditionalGeneration.from_pretrained("models/v4")
# model = TFT5ForConditionalGeneration.from_pretrained("ckpts/v4/ckpt-0")


SyntaxError: trailing comma not allowed without surrounding parentheses (660884058.py, line 1)

In [ ]:
def load_jsonl(path):
    with open(path) as input_file:
        lines = list(map(json.loads, input_file))

    return lines

def load_data(root_path):
    files = os.listdir(root_path)
    dataset= {}

    for filename in files:
        filepath = root_path + f'/{filename}'

        dataset[filename] = load_jsonl(filepath)

    return dataset

In [ ]:
dataset = load_data('data/L1')

In [ ]:
models = ['llama3.1', 'gemma2:2b', 'gemma2']

test_from = 'attrebute_val'
train_from = 'attrebute_train'
target_from = 'attrebute_test'

train_inputs = train_from + '.data'
train_labels = train_from + '.solution'

test_inputs = test_from + '.data'
test_labels = test_from + '.solution'

target_inputs = target_from + '.data'

test_inputs = dataset[test_inputs]
test_labels = dataset[test_labels]
train_inputs = dataset[train_inputs]
train_labels = dataset[train_labels]
target_inputs = dataset[target_inputs]

In [ ]:
SEQ_LEN=250
prompt = {
    "label": "The product belongs to these categories:\nBrand: {details_Brand}</s>Level 0: {L0_category}</s>Level 1: {L1_category}</s>Level 2: {L2_category}</s>Level 3: {L3_category}</s>Level 4: {L4_category}</s>",
    # "label": "The product belongs to these categories:\nBrand: {details_Brand}\nLevel 0: {L0_category}\nLevel 1: {L1_category}\nLevel 2: {L2_category}\nLevel 3: {L3_category}\nLevel 4: {L4_category}",
    "input": "Categories the product into 5 levels of categories and identify the brand:\nTitle: {title}\nStore:{store}\nManufacturer:{details_Manufacturer}"
}

def make_prompt(data):
    if 'title' in data:
        input_type = 'input'
    else:
        input_type = 'label'

    return prompt[input_type].format(**data)

def batch_make_prompt(data):
    return [make_prompt(item) for item in data]
# def tokenize()

In [ ]:
print(prompt['label'])

In [ ]:
train_input_prompts = pqdm(train_inputs, make_prompt, n_jobs=5)
train_label_prompts = pqdm(train_labels, make_prompt, n_jobs=5)
test_input_prompts = pqdm(test_inputs, make_prompt, n_jobs=5)
test_label_prompts = pqdm(test_labels, make_prompt, n_jobs=5)

In [4]:
def train_model(inputs, labels, return_output=False):
    inputs = [item.decode() for item in inputs]
    labels = [item.decode() for item in labels]

    encoding = tokenizer(
        inputs,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

    target_encoding = tokenizer(
        labels,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    labels = target_encoding.input_ids
    labels[labels == tokenizer.pad_token_id] = -100

    preds = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    if not return_output:
        return preds.loss
    
    return preds, preds.loss

def predict(inputs):
    

    encoding = tokenizer.batch_encode_plus(
        inputs,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
    outputs = model.generate(input_ids, max_new_tokens=SEQ_LEN)

    return outputs

In [5]:
BATCH_SIZE = 25
train_ds = tf.data.Dataset.from_tensor_slices((train_input_prompts, train_label_prompts)).shuffle(1000).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((test_input_prompts, test_label_prompts)).shuffle(1000).batch(BATCH_SIZE*4)

NameError: name 'train_input_prompts' is not defined

In [6]:
unified_input_prompts = train_input_prompts.extend(test_input_prompts)
unified_label_prompts = train_label_prompts.extend(test_label_prompts)
train_ds = tf.data.Dataset.from_tensor_slices((train_input_prompts, train_label_prompts)).shuffle(1000).batch(BATCH_SIZE)

NameError: name 'train_input_prompts' is not defined

In [ ]:
# with strategy.scope():
optimizer = keras.optimizers.AdamW(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=model.compute_loss)

In [ ]:
model.summary()

In [ ]:
EPOCHS=4

per_epoch_loss=[]

# with strategy.scope():
for e in range(EPOCHS):
    dataset_iter = train_ds.as_numpy_iterator()

    progbar = Progbar(len(train_ds), stateful_metrics=['train_loss'])
    loss_metric = Mean("train_loss")

    for i, batch in enumerate(dataset_iter):
        inputs, labels = batch

        with tf.GradientTape() as tape:
            loss = train_model(inputs, labels)

        grads = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss_metric.update_state(loss)
        values = [('train_loss', loss_metric.result())]

        progbar.update(i+1, values=values)

    model.save_pretrained(f'./ckpts/v5/ckpt-{e}')
    

In [ ]:
model.save_pretrained('./models/v4')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_metrics(y_true, y_preds):
    return (
        accuracy_score(y_true, y_preds),
        precision_score(y_true, y_preds, average='macro', zero_division=0),
        recall_score(y_true, y_preds, average='macro', zero_division=0),
        f1_score(y_true, y_preds, average='macro', zero_division=0)
    )

import re

def extract_categories(text):
    key_map = {
        'Brand': 'details_Brand',
        'Level 0': 'L0_category',
        'Level 1': 'L1_category',
        'Level 2': 'L2_category',
        'Level 3': 'L3_category',
        'Level 4': 'L4_category'
    }
    
    # Regex patterns to match each key
    patterns = {
        'Brand': r'Brand:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 0': r'Level 0:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 1': r'Level 1:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 2': r'Level 2:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 3': r'Level 3:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 4': r'Level 4:\s*(.+?)(?=\s+Level|\s*<\/s>)'
    }
    
    # Extracting data using the patterns
    extracted_data = {v: 'na' for v in key_map.values()}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            extracted_data[key_map[key]] = match.group(1).strip()
    
    return extracted_data


In [ ]:
# dataset_iter = test_ds.as_numpy_iterator()

# progbar = Progbar(len(test_ds), stateful_metrics=['val_loss'])
# loss_metric = Mean("val_loss")

# for i, batch in enumerate(dataset_iter):
#     inputs, labels = batch

#     loss = train_model(inputs, labels)

#     loss_metric.update_state(loss)
#     values = [('val_loss', loss_metric.result())]

#     progbar.update(i+1, values=values)

In [ ]:
# dataset_iter = test_ds.as_numpy_iterator()

# progbar = Progbar(len(test_ds), stateful_metrics=['val_loss'])
# acc_metric = Mean("val_accuracy")

# for i, batch in enumerate(dataset_iter):
#     inputs, labels = batch
#     labels = list(map(extract_categories, [item.decode() + '</s>' for item in labels]))

#     preds = predict([item.decode() for item in inputs])
#     preds = tokenizer.batch_decode(preds)
#     preds = list(map(extract_categories, preds))

#     labels = [list(item.values()) for item in labels]
#     preds = [list(item.values()) for item in preds]

#     labels = np.array(labels)
#     preds = np.array(preds)

#     TP = np.sum(labels == preds)
#     FP = np.sum(labels != preds)

#     accuracy = np.sum(np.all(labels == preds, axis=1))/len(labels)

#     acc_metric.update_state(accuracy)
#     values = [('val_accuracy', acc_metric.result())]

#     progbar.update(i+1, values=values)

In [ ]:
target_prompts = pqdm(target_inputs, make_prompt, n_jobs=5)

In [ ]:
tokenizer.decode(tokenizer(train_input_prompts[0]).input_ids)

In [ ]:
def tokenizer_data(item):
    item = item.numpy().decode('utf-8')
    encoding = tokenizer.encode(
        item,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )
    return encoding[0]

def batch_tokenize(batch):
    encoding = tokenizer.batch_encode_plus(
        batch,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )
    return encoding.input_ids

In [ ]:
def chunk_list(lst, n):
    # Split the list into chunks of n items
    chunks = [lst[i:i + n] for i in range(0, len(lst), n)]
    return chunks

def add_prefix(text, prefix):
    return text + prefix

In [ ]:
batches = chunk_list(target_prompts, n=512)
batches = pqdm(batches, batch_tokenize, n_jobs=12)

In [ ]:
input_ids = batches[0]
prefix = ['The product belongs to these categories:\nBrand:'] * input_ids.shape[0]
prefix = tokenizer.batch_encode_plus(prefix, return_tensors='np', add_special_tokens=False).input_ids
outputs = model.generate(
    input_ids,
    decoder_input_ids=prefix,
    max_new_tokens=10, 
    num_beams=1,
    early_stopping=True)

In [ ]:
answers = []

prefix = ['The product belongs to these categories:\nBrand:'] * input_ids.shape[0]
prefix = tokenizer.batch_encode_plus(prefix, return_tensors='np', add_special_tokens=False).input_ids

for input_ids in tqdm(batches):
    outputs = model.generate(
        input_ids, 
        decoder_input_ids=prefix,
        max_new_tokens=SEQ_LEN, 
        num_beams=1,           # Setting num_beams=1 ensures greedy decoding
        early_stopping=True)

    answers.extend(tokenizer.batch_decode(outputs))

In [ ]:
# import regex as re
# word=r'.'
# pattern = rf"(Brand:\s?{word}+)? (Level 0:\s?{word}+)? (Level 1:\s?{word}+)? (Level 2:\s?{word}+)? (Level 3:\s?{word}+)? (Level 4:\s?{word}+)?"

# def extract_categories(output):

#     brand, l0, l1, l2, l3, l4 = re.findall(pattern, output)[0]
#     return {
#         'details_Brand': brand,
#         'L0_category': l0,
#         'L1_category': l1,
#         'L2_category': l2,
#         'L3_category': l3,
#         'L4_category': l4
#     }
# print(pattern)


In [ ]:
final_outputs = []
for i, item in enumerate(tqdm(answers)):
    details = {'indoml_id': i}
    details.update(extract_categories(item))

    final_outputs.append(json.dumps(details) + '\n')

In [ ]:
output_path = './output/L4/'

with open(output_path + 'attribute_test_topjourney4.predict', 'w') as output_file:
    output_file.writelines(final_outputs)